In [2]:
import os
import pandas as pd
import razdel
import re
import random
import json
from tqdm.auto import tqdm, trange

In [7]:
DATA_PATH_PREFIX = '../parsing_aligned_data/results/'

## Load sentence-parallel data

In [8]:
bible_df = pd.read_csv(DATA_PATH_PREFIX + 'moksha_bible.tsv', sep='\t')
print(bible_df.shape)

bible_df = bible_df.dropna()
print(bible_df.shape)

bible_df = bible_df[~bible_df['mdf'].str.startswith('Глава ')]
print(bible_df.shape)
print(bible_df.columns)

(11650, 4)
(11633, 4)
(11583, 4)
Index(['Unnamed: 0', 'mdf', 'ru', 'source'], dtype='object')


In [9]:
assert not bible_df.isna().sum().sum()

In [10]:
bible_pairs = list(zip(bible_df['mdf'], bible_df['ru']))
print(len(bible_pairs))

11583


In [11]:
random.choice(bible_pairs)

('И ули тяфта: эрсь, кие тердьсы Азорть лемонц, ули идеф .',
 'И будет: всякий, кто призовет имя Господне, спасется.')

In [12]:
word_df = pd.read_csv(DATA_PATH_PREFIX + 'all_dicts_data.tsv', sep='\t')

In [16]:
assert not word_df.isna().sum().sum()

In [17]:
word_pairs = list(zip(word_df['mdf'], word_df['ru']))
print(len(word_pairs))

3589


In [18]:
random.choice(word_pairs)

('зверь', 'зверь')

In [20]:
phrases_df = pd.read_csv(DATA_PATH_PREFIX + 'all_phrases.tsv', sep='\t')

In [21]:
assert not phrases_df.isna().sum().sum()

In [22]:
phrases_pairs = list(zip(phrases_df['mdf'], phrases_df['ru']))
print(len(phrases_pairs))

726


In [23]:
random.choice(phrases_pairs)

('Здесь много народу', 'Тяса лама ломаньда')

In [24]:
all_pairs = sorted({
    c for c in
    word_pairs + phrases_pairs + bible_pairs
    if c[0] and c[1]
})
print(len(all_pairs))

15898


In [25]:
# funcs for fixing breaks in books "по-\ нимаемый иска- женный при- мер" -> "понимаемый искаженный пример"

def dashrepl(matchobj):
    return matchobj.group(1) + matchobj.group(3)

def fix_breaks(text):
    return re.sub('([а-яА-ЯёЁ])([\-+]\s+)([а-яА-ЯёЁ])', dashrepl, text)

In [26]:
def fix_text(text):
    text = re.sub('\s+', ' ', text).strip()
    text = fix_breaks(text)
    return text

In [33]:
for i, [t1, t2] in enumerate(all_pairs):
    all_pairs[i] = (fix_text(t1), fix_text(t2))

In [34]:
for _ in range(10):
    print(random.choice(all_pairs))

('Монь койсон, тяфта кальдявста.', 'По-моему, так плохо.')
('Мзярда аделавсть озксвятемань шинза, сон мрдась эсь кудозонза.', 'А когда окончились дни службы его, возвратился в дом свой.')
('Но Иисус кемоста кардазень тянь колга киндинге корхнемс.', 'Но Он строго приказал им никому не говорить о сем,')
('Иисус каршезонза мярьгсь: Вай, тяниень пингонь аф кемай ломатть! Мзярс Мон карман улема тинь мархтонт? И мзярс карман кирдемонт тинь? Вятесть цёранять Тейне.', 'Отвечая ему, Иисус сказал: о, род неверный! доколе буду с вами? доколе буду терпеть вас? Приведите его ко Мне.')
('Тяфта сонь лацонза шамафксонь канникс пряснон вятезь ляды кемай еврейхневок, нльне Варнававок шоворсь ёткозост.', 'Вместе с ним лицемерили и прочие Иудеи, так что даже Варнава был увлечен их лицемерием.')
('Озондода тянксонга: катк мон панчса сонь смузенц тяфта, кода тейне эряви тянь тиемс.', 'дабы я открыл ее, как должно мне возвещать.')
('шабат', 'дети')
('Шкайсь мярьгсь Авраамонди: Рьвяцень Сарань оду ни тяк лемн

# Select traindev data

In [35]:
print(len(set(bible_pairs)))

random.seed(1)
bible_devtest = random.sample(sorted(set(bible_pairs)), k=800)

11583


In [ ]:
# get some other sources

In [36]:
components = {
    'bible': bible_devtest,
    # other scources
}
dev = {k: v[:len(v)//2] for k, v in components.items()}
test = {k: v[len(v)//2:] for k, v in components.items()}

In [37]:
print(len({s for c in dev.values() for s in c}))
print(len({s for c in test.values() for s in c}))

400
400


In [41]:
with open('train_test_splitting/dev_v1.json', 'w') as f:
    json.dump(dev, f, indent=2, ensure_ascii=False)

In [40]:
with open('train_test_splitting/test_v1.json', 'w') as f:
    json.dump(test, f, indent=2, ensure_ascii=False)

In [38]:
devtest = {s for c in components.values() for s in c}
len(devtest)

800

In [39]:
train = sorted(set(all_pairs).difference(devtest))
print(len(train))

15118


In [42]:
with open('train_test_splitting/train_v1.json', 'w') as f:
    json.dump(train, f, indent=2, ensure_ascii=False)

# Prepare the whole crawled corpus

In [336]:
import os
import pandas as pd
import razdel
import re
import random
import json
from tqdm.auto import tqdm, trange

In [690]:
def replace_repeating_char(text, max_run=3):
    res = []
    prev = None
    run = 0
    for c in text:
        if c != prev:
            run = 0
        run += 1
        if run <= max_run:
            res.append(c)
        prev = c
    return ''.join(res)

replace_repeating_char("' 2768 [8_________________________ 2400 3 сядт, 4 дес. 6 един.")

"' 2768 [8___ 2400 3 сядт, 4 дес. 6 един."

In [691]:
def final_clean(text):
    text = fix_text(text)
    text = replace_repeating_char(text)
    if max(len(w) for w in text.split()) < 2:
        return ''
    
    if not re.match('.*[а-яё].*', text.lower()):
        return ''
    text = re.sub('(\. )+', '. ', text)
    
    return text.strip()

In [692]:
all_sentences = sorted({
    final_clean(s) for s in set(
        ...
    )
}.union({t[0] for t in train}).difference(devtest_er).difference({''}))
print(len(all_sentences))

333651


In [711]:
for _ in range(10):
    print(random.choice(all_sentences))
    print('------')

1931—1932 иетнесэ ульнесь татронь эйкакшонь журналонь редакторокс, конань нолдась ЦК ВЛКСМ.
------
Сдано в производство 9/VI-40 г. Подписано к печати 13/VIII-40 г. Формат бум.
------
Юткось аволь покш — вете вайгельпеть.
------
атна ломань
------
Те арсевкстэнть кармась содамо Паволонь сазоронзо цёра. Сон мольсь ушмокудов ды ёвтась теде лелянстэнь.
------
И сон аф содасы, кода синь явомс.
------
сюдома
------
пургондамс
------
Роботась мольсь вадрясто.
------
А кувать кортасть — кедьёнкс мельга тусть: Лия велетнень ланга кучокшность Кулянь пачтицят, бойка чийницят.
------


### todo: clean by perplexity 

In [694]:
with open('corpus/myv_mono.v1.json', 'w') as f:
    json.dump(all_sentences, f, indent=2, ensure_ascii=False)